# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [38]:
import logging
import os
import csv
import json

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails

from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

from azureml.automl.runtime.onnx_convert import OnnxConverter


# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.49.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()
experiment_name = 'cancer-auto-ml'
project_folder = './capstone-project'

experiment=Experiment(ws, experiment_name)
experiment

Performing interactive authentication. Please follow the instructions on the terminal.
Interactive authentication successfully completed.


To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RMCLY2WPW to authenticate.


Name,Workspace,Report Page,Docs Page
cancer-auto-ml,quick-starts-ws-233079,Link to Azure Machine Learning studio,Link to Documentation


In [4]:

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "cluster-project"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [8]:
dataset_name = 'cancer-data'
try: 
    ds = ws.datasets[dataset_name]
except KeyError:
    print("Dataset not found, create and rerun this cell!")
    raise

In [10]:
df = ds.to_pandas_dataframe()
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [11]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=ds,
                             label_column_name="diagnosis",   
                             path = project_folder,
                             enable_early_stopping=True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [12]:

remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
cancer-auto-ml,AutoML_72024c89-fcd5-47e9-b812-3889f402a6be,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [13]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
cancer-auto-ml,AutoML_72024c89-fcd5-47e9-b812-3889f402a6be,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  In order to accurately evaluate the model(s) trained by AutoML, we leverage a dataset that the model is not trained on. Hence, if the user doesn't provide an explicit validation dataset, a part of the training dataset is used to achieve this. For smaller datasets (fewer than 20,000 samples), cross-validation is leveraged, else a single hold-out set is split from the training data to serve as the validation dataset. Hence, for your input data we leverage cross-validation with 10 folds, if the number of training samples are fewer than 1000, and 3 folds in all other cases.
              Learn mo

{'runId': 'AutoML_72024c89-fcd5-47e9-b812-3889f402a6be',
 'target': 'cluster-project',
 'status': 'Completed',
 'startTimeUtc': '2023-05-08T08:44:29.902461Z',
 'endTimeUtc': '2023-05-08T09:01:43.823901Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'cluster-project',
  'AMLSettingsJsonString': '{"path":null,"name":"cancer-auto-ml","subscription_id":"6971f5ac-8af1-446e-8034-05acea24681f","resource_group":"aml-quickstarts-233079","workspace_name":"quick-starts-ws-233079","region":"westus2","compute_target":"cluster-p

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [45]:

best_run = remote_run.get_best_child()
print(best_run)

brm = best_run.get_metrics()

for metric_name, metric in zip(brm.keys(), brm.values()):
    print(f"{metric_name}: {metric}")

Run(Experiment: cancer-auto-ml,
Id: AutoML_72024c89-fcd5-47e9-b812-3889f402a6be_38,
Type: azureml.scriptrun,
Status: Completed)
recall_score_micro: 0.9806390977443609
AUC_micro: 0.996477644612785
matthews_correlation: 0.9599511929248765
average_precision_score_weighted: 0.9974336038937496
average_precision_score_macro: 0.9973197531232267
precision_score_macro: 0.9816561703403808
f1_score_weighted: 0.9805604598289495
f1_score_macro: 0.9793311427258
f1_score_micro: 0.9806390977443609
precision_score_micro: 0.9806390977443609
average_precision_score_micro: 0.9966146108956895
recall_score_macro: 0.9783720321492062
accuracy: 0.9806390977443609
AUC_weighted: 0.9971575796326435
precision_score_weighted: 0.9817039608102791
AUC_macro: 0.9971575796326435
weighted_accuracy: 0.9827143255560973
log_loss: 0.08682214970430088
norm_macro_recall: 0.9567440642984121
recall_score_weighted: 0.9806390977443609
balanced_accuracy: 0.9783720321492062
confusion_matrix: aml://artifactId/ExperimentRun/dcid.AutoM

In [30]:
print(best_run.properties)

{'runTemplate': 'automl_child', 'pipeline_id': '__AutoML_Ensemble__', 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'AUC_weighted\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'cancer-auto-ml\',\'compute_target\':\'cluster-project\',\'subscription_id\':\'6971f5ac-8af1-446e-8034-05acea24681f\',\'region\':\'westus2\',\'spark_service\':None}","ensemble_run_id":"AutoML_72024c89-fcd5-47e9-b812-3889f402a6be_38","experiment_name":"cancer-auto-ml","workspace_name":"quick-starts-ws-233079","subscription_id":"6971f5ac-8af1-446e-8034-05acea24681f","resource_group_name":"aml-quickstarts-233079"}}]}', 'training_percent': '100', 'predicted_cost': None, 'iteration': '38', '_aml_system_scenario_identification': 'Remote.Child', '_azureml.ComputeTargetType': '

In [34]:
model_name = best_run.properties["model_name"]

script_file_name = "inference/score.py"

best_run.download_file("outputs/scoring_file_v_1_0_0.py", "inference/score.py")

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [37]:
description = "AutoML Model trained on cancer data to predict if a patient has a malignant or benign tumour"
tags = None
model = remote_run.register_model(
    model_name=model_name, description=description, tags=tags
)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [43]:

inference_config = InferenceConfig(entry_script=script_file_name)

aciconfig = AciWebservice.deploy_configuration(
    cpu_cores=2,
    memory_gb=2,
    tags={"area": "CancerData", "type": "automl_cd_classification"},
    description="sample service for CancerData Automl Classification",
)

aci_service_name = model_name.lower()
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

automl72024c89f38
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-05-08 10:23:28+00:00 Creating Container Registry if not exists.
2023-05-08 10:23:29+00:00 Generating deployment configuration.
2023-05-08 10:23:29+00:00 Submitting deployment to compute.
2023-05-08 10:23:34+00:00 Checking the status of deployment automl72024c89f38..
2023-05-08 10:25:27+00:00 Checking the status of inference endpoint automl72024c89f38.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


TODO: In the cell below, print the logs of the web service and delete the service

In [44]:
aci_service.get_logs()

'2023-05-08T10:25:24,750195000+00:00 - rsyslog/run \n2023-05-08T10:25:24,757407300+00:00 - gunicorn/run \n2023-05-08T10:25:24,760238300+00:00 | gunicorn/run | \n2023-05-08T10:25:24,765803500+00:00 | gunicorn/run | ###############################################\n2023-05-08T10:25:24,770333200+00:00 | gunicorn/run | AzureML Container Runtime Information\n2023-05-08T10:25:24,773734800+00:00 | gunicorn/run | ###############################################\n2023-05-08T10:25:24,779430400+00:00 | gunicorn/run | \n2023-05-08T10:25:24,783996000+00:00 - nginx/run \n2023-05-08T10:25:24,786343800+00:00 | gunicorn/run | \n2023-05-08T10:25:24,802297400+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20230412.v4\n2023-05-08T10:25:24,803679500+00:00 | gunicorn/run | \n2023-05-08T10:25:24,805015200+00:00 | gunicorn/run | \n2023-05-08T10:25:24,811638800+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
